In [21]:
import yfinance as yf
import requests
import numpy as np
import pandas as pd
import nbimporter
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [9]:
from Financial_Data import search_stock_symbol,symbol_changes,pe_and_market_cap,rsi,analyze_rsi,macd,interpret_macd,fibonacci_retracement,fib_insight,calculate_trend,analyze_trend

In [10]:
def calculate_signal(symbol, data, current_date):
    # Slice the data up to the current date
    current_data = data.loc[:current_date]
    
    # Get the latest info
    comp = yf.Ticker(symbol)
    info = comp.info
    
    curr_price = current_data['Close'].iloc[-1]
    
    weights = {
    'pe': 0.5,
    'rsi': 0.8,
    'macd': 0.8,
    'fib': 0.2,
    'trend': 0.6
}
    
    # PE RATIO AND CAP ID
    market_cap, pe, is_large_cap = pe_and_market_cap(info)
    
    if not is_large_cap:
        return 0  
    
    if pe <= 20:
        pe_score = 1  # Buy
    else:
        pe_score = -1  # Sell
    
    # RSI
    r, window = rsi(current_data)
    rsi_score, _ = analyze_rsi(r)
    
    # MACD
    ma = macd(current_data)
    macd_score, _ = interpret_macd(ma)
    
    # Fibonacci Retracement
    fib = fibonacci_retracement(current_data)
    current_price = current_data['Close'].iloc[-1]
    prev_price = current_data['Close'].iloc[-2]
    trend = fib['trend']
    levels = {
        'retracement': fib['retracement_levels'],
        'extension': fib['extension_levels']
    }
    _, fib_score = fib_insight(current_price=current_price, trend=trend, levels=levels,previous_price=prev_price)
    
    data_trend = calculate_trend(data, short_window=14, long_window=50)
    trend_score, _ = analyze_trend(data_trend)
    
    final_score = (
    (pe_score * weights['pe']) + 
    (rsi_score * weights['rsi']) + 
    (macd_score * weights['macd']) + 
    (fib_score * weights['fib']) +
    (trend_score * weights['trend']))
    
    if final_score > -0.3 and final_score < 0.3:
        ans = 0
        
    elif final_score >= 0.3:
        ans = 1
        
    elif final_score <= -0.3:
        ans = -1
    
    return ans

In [11]:
def backtest(symbol, initial_balance=10000, holding_period=5, stop_loss=-5, take_profit=10, future_window=5):
    # Download data
    stock = yf.Ticker(symbol)
    data = stock.history(period='2y', interval='1d')
    
    # Initialize variables
    position = 0
    balance = initial_balance
    shares = 0
    trades = []
    good_predictions = 0
    bad_predictions = 0
    
    # Iterate through each day
    for i, current_date in enumerate(data.index[:-future_window]):
        current_price = data.loc[current_date, 'Close']
        
        if len(data.loc[:current_date]) < 60:  # Skip the first 60 days to have enough data for indicators
            continue
        
        # Get the signal
        signal = calculate_signal(symbol, data, current_date)
        
        # Check if we need to sell due to stop loss, take profit, or holding period
        if position == 1:
            days_held = (current_date - trade_entry_date).days
            price_change = (current_price - entry_price) / entry_price * 100
            
            if price_change <= stop_loss or price_change >= take_profit or days_held >= holding_period:
                # Sell
                balance += shares * current_price
                trade_type = 'sell'
                trades.append((trade_type, current_date, current_price, shares, balance))
                
                future_price = data['Close'].iloc[i + future_window]
                if (signal == 1 and future_price > current_price) or (signal == -1 and future_price < current_price):
                    good_predictions += 1
                else:
                    bad_predictions += 1
                
                shares = 0
                position = 0
        
        # Execute trades based on signal
        if signal == 1 and position == 0:  # Buy signal
            shares_to_buy = balance // current_price
            cost = shares_to_buy * current_price
            balance -= cost
            shares += shares_to_buy
            position = 1
            trade_type = 'buy'
            trades.append((trade_type, current_date, current_price, shares_to_buy, balance))
            entry_price = current_price
            trade_entry_date = current_date
            
            # Check if the buy prediction was good
            future_price = data['Close'].iloc[i + future_window]
            if future_price > current_price:
                good_predictions += 1
            else:
                bad_predictions += 1
            
        elif signal == -1 and position == 1:  # Sell signal
            balance += shares * current_price
            trade_type = 'sell'
            trades.append((trade_type, current_date, current_price, shares, balance))
            
            # Check if the sell prediction was good
            future_price = data['Close'].iloc[i + future_window]
            if future_price < current_price:
                good_predictions += 1
            else:
                bad_predictions += 1
            
            shares = 0
            position = 0
    
    # Final value
    final_balance = balance + shares * data['Close'].iloc[-1]
    return_pct = (final_balance - initial_balance) / initial_balance * 100
    
    total_predictions = good_predictions + bad_predictions
    prediction_accuracy = (good_predictions / total_predictions * 100) if total_predictions > 0 else 0
    
    return {
        'final_balance': final_balance,
        'return_pct': return_pct,
        'trades': trades,
        'good_predictions': good_predictions,
        'bad_predictions': bad_predictions,
        'prediction_accuracy': prediction_accuracy
    }

In [30]:
symbols = ['RELIANCE.NS','TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'HINDUNILVR.NS', 'BHARTIARTL.NS', 'ICICIBANK.NS','SBIN.NS', 'LICI.NS', 'ITC.NS'] 
results = {}
for symbol in tqdm(symbols, desc="Backtesting symbols", unit="symbol"):
    result = backtest(symbol, holding_period=26, future_window=26, initial_balance = 100000)
    results[symbol] = result

# Analyze results
for symbol, result in results.items():
    print(f"\nResults for {symbol}:")
    print(f"Final Balance: Rs{result['final_balance']:.2f}")
    print(f"Return: {result['return_pct']:.2f}%")
    print(f"Number of trades: {len(result['trades'])}")
    print(f"Good trades: {result['good_predictions']}")
    print(f"Bad trades: {result['bad_predictions']}")
    print(f"Accuracy: {result['prediction_accuracy']:.2f}%")

Backtesting symbols:   0%|          | 0/10 [00:00<?, ?symbol/s]


Results for RELIANCE.NS:
Final Balance: Rs102652.35
Return: 2.65%
Number of trades: 12
Good trades: 7
Bad trades: 5
Accuracy: 58.33%

Results for TCS.NS:
Final Balance: Rs110401.89
Return: 10.40%
Number of trades: 28
Good trades: 16
Bad trades: 12
Accuracy: 57.14%

Results for HDFCBANK.NS:
Final Balance: Rs106929.27
Return: 6.93%
Number of trades: 53
Good trades: 26
Bad trades: 27
Accuracy: 49.06%

Results for INFY.NS:
Final Balance: Rs136486.26
Return: 36.49%
Number of trades: 52
Good trades: 27
Bad trades: 25
Accuracy: 51.92%

Results for HINDUNILVR.NS:
Final Balance: Rs94561.17
Return: -5.44%
Number of trades: 52
Good trades: 27
Bad trades: 25
Accuracy: 51.92%

Results for BHARTIARTL.NS:
Final Balance: Rs137696.22
Return: 37.70%
Number of trades: 63
Good trades: 32
Bad trades: 31
Accuracy: 50.79%

Results for ICICIBANK.NS:
Final Balance: Rs131469.35
Return: 31.47%
Number of trades: 85
Good trades: 45
Bad trades: 40
Accuracy: 52.94%

Results for SBIN.NS:
Final Balance: Rs167774.41
R

In [33]:
for trade in results['RELIANCE.NS']['trades']:
            print(f"{trade[0].capitalize()} on {trade[1]}: Price Rs {trade[2]:.2f}, Shares: {trade[3]}, Balance: {round(trade[4],2)}")

Buy on 2023-02-09 00:00:00+05:30: Price Rs 2159.65, Shares: 46.0, Balance: 656.16
Sell on 2023-02-22 00:00:00+05:30: Price Rs 2180.59, Shares: 46.0, Balance: 100963.48
Buy on 2023-03-03 00:00:00+05:30: Price Rs 2186.55, Shares: 46.0, Balance: 382.08
Sell on 2023-03-09 00:00:00+05:30: Price Rs 2162.58, Shares: 46.0, Balance: 99860.85
Buy on 2023-10-12 00:00:00+05:30: Price Rs 2341.45, Shares: 42.0, Balance: 1519.82
Sell on 2023-10-16 00:00:00+05:30: Price Rs 2336.12, Shares: 42.0, Balance: 99636.92
Buy on 2023-11-01 00:00:00+05:30: Price Rs 2289.63, Shares: 43.0, Balance: 1182.88
Sell on 2023-11-21 00:00:00+05:30: Price Rs 2370.85, Shares: 43.0, Balance: 103129.57
Buy on 2024-06-07 00:00:00+05:30: Price Rs 2929.96, Shares: 35.0, Balance: 581.12
Sell on 2024-06-19 00:00:00+05:30: Price Rs 2907.43, Shares: 35.0, Balance: 102341.26
Buy on 2024-08-19 00:00:00+05:30: Price Rs 2976.80, Shares: 34.0, Balance: 1130.05
Sell on 2024-09-05 00:00:00+05:30: Price Rs 2985.95, Shares: 34.0, Balance: 1

Results for RELIANCE.NS:
Final Balance: Rs10251.35
Return: 2.51%
Number of trades: 12
Good trades: 7
Bad trades: 5
Accuracy: 58.33%

Results for TCS.NS:
Final Balance: Rs11080.33
Return: 10.80%
Number of trades: 28
Good trades: 16
Bad trades: 12
Accuracy: 57.14%

Results for HDFCBANK.NS:
Final Balance: Rs10655.21
Return: 6.55%
Number of trades: 53
Good trades: 26
Bad trades: 27
Accuracy: 49.06%

Results for INFY.NS:
Final Balance: Rs13305.38
Return: 33.05%
Number of trades: 52
Good trades: 27
Bad trades: 25
Accuracy: 51.92%

Results for HINDUNILVR.NS:
Final Balance: Rs9588.53
Return: -4.11%
Number of trades: 52
Good trades: 27
Bad trades: 25
Accuracy: 51.92%

Results for BHARTIARTL.NS:
Final Balance: Rs13705.05
Return: 37.05%
Number of trades: 63
Good trades: 32
Bad trades: 31
Accuracy: 50.79%

Results for ICICIBANK.NS:
Final Balance: Rs12989.89
Return: 29.90%
Number of trades: 85
Good trades: 45
Bad trades: 40
Accuracy: 52.94%

Results for SBIN.NS:
Final Balance: Rs16665.14
Return: 66.65%
Number of trades: 63
Good trades: 33
Bad trades: 30
Accuracy: 52.38%

Results for LICI.NS:
Final Balance: Rs12508.73
Return: 25.09%
Number of trades: 59
Good trades: 26
Bad trades: 33
Accuracy: 44.07%

Results for ITC.NS:
Final Balance: Rs11321.08
Return: 13.21%
Number of trades: 30
Good trades: 16
Bad trades: 14
Accuracy: 53.33%

In [28]:
avg26 = 0
for i in symbols:
    avg26 += results[i]['prediction_accuracy']

avg26 /= len(symbols)
avg26

52.18958576846235

In [29]:
avg26 = 0
for i in symbols:
    avg26 += results[i]['return_pct']

avg26 /= len(symbols)
avg26

22.070690216064456

In [31]:
avg26 = 0
for i in symbols:
    avg26 += results[i]['return_pct']

avg26 /= len(symbols)
avg26

22.737814157104495